In [1]:
%load_ext autotime

time: 0 ns (started: 2022-09-23 23:01:46 -05:00)


# MODULES

In [2]:
import panel as pn
import panel.widgets as pnw
pn.extension('plotly')

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

import numpy as np
import math

import pandas as pd

from scipy.stats import wrapcauchy
from scipy.stats import levy_stable
from scipy.spatial import distance

time: 2.69 s (started: 2022-09-23 23:01:46 -05:00)


# CLASESS

## Vec2d

In [3]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:            
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y
            
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)
    
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    # rotate vector
    def rotated(self, angle):        
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

time: 0 ns (started: 2022-09-23 23:01:49 -05:00)


## Explorers

In [4]:
class Explorers(object):
    
    # This class substitutes BM trajectory and CREW Trajectory
    def __init__(self, x = 0, y = 0):
        self.x = x
        self.y = y
        
    def trace_trajectories(self, n_traj = 1, traj_type= "BM_2d", n_steps =1000, speed = 6, crw_exponents = [0.6]):
        """
            Generates the trajectories of the explorers according to their type
            Arguments: 
                traj_type:
                n_steps:
                speed:
                n_traj:
                crw_exponents:
          """
        
        trajectories_df = pd.DataFrame(columns=['x','y','traj'])
        

        for j in range(n_traj):
            
            velocity = Vec2d(speed, 0)
            
            
            traj_aux_array = np.ones((n_steps,3))*[self.x,self.y,0]
            traj_aux_array[0,2] = j
            
            crw_ex = lambda : crw_exponents[j] if len(crw_exponents) > 1 else crw_exponents[0]
            exponent = crw_ex()

            for i in range(1,n_steps):
                turn_angle = self.generate_angle(traj_type, exponent)
                
                velocity = velocity.rotated(turn_angle)

                traj_aux_array[i] = traj_aux_array[i-1,:]+[velocity.x,velocity.y,0]
                traj_aux_array[i,2] = j

                
            temp_df = pd.DataFrame(data = traj_aux_array, columns=['x','y','traj'])
            
            trajectories_df = pd.concat([trajectories_df,temp_df], ignore_index=True)
          
        return trajectories_df
    
    
    
    
    # angle choice 
    def generate_angle(self,traj_type, exponent = 0.6):
        angle = 0
        if traj_type == "BM_2d":
            angle = np.random.uniform(low=-np.pi, high=np.pi)
        
        if traj_type == "BM_nsew":
            angle = np.random.choice([0,np.pi/2, np.pi, 3*np.pi])
        
        if traj_type == "CRW":
            angle = wrapcauchy.rvs(exponent)
            
            
        return angle

time: 0 ns (started: 2022-09-23 23:01:49 -05:00)


## Levy Flight

In [5]:
class LFlyTrajectory(object):
    
    def __init__(self, x = 0 , y = 0):
        self.x = 0
        self.y = 0
        
        
    def levy_walk(self, alpha = 1, beta = 1, loc = 6 , speed = 3, samples = 100000):
        # Init velocity vector

        velocity = Vec2d(speed,0)

        # Init df
        LW_df = pd.DataFrame(columns=['x','y','traj'])
        lw_3d = np.array([[0,0,0]])
        aux = np.array([[0,0,0]])

        for i in range(1,samples):
            # get random n_steps form levy distribution
            step_size = levy_stable.rvs(alpha, beta, loc)
            step_size = int(np.ceil(abs(step_size)))

            theta = wrapcauchy.rvs(c=0.7, loc=loc)

            # update velocity
            velocity = velocity.rotated(theta)

            for j in range(step_size):
                row = len(lw_3d)
                aux = np.array([lw_3d[row-1,:]+[velocity.x,velocity.y,0]])

                lw_3d = np.vstack([lw_3d,aux])

            if(len(lw_3d) >= samples):
                break

        LW_df = pd.DataFrame(data = lw_3d, columns=['x','y','traj'])

        return LW_df;

time: 0 ns (started: 2022-09-23 23:01:49 -05:00)


## Metrics

### Path lenght

In [6]:
class TrajectoryMetrics():
    
    
    def get_path_lengt(self, trajectories, traj_df):
        
        path_lenghts = pd.DataFrame(columns=['distance','traj'])
            
        for i in range(n_traj):
            aux_df= trajectories.loc[trajectories.traj == i, ['x','y']]

            #Calculate lenghts for given trajectory
            euc_dis = np.array([get_euclidean_distance(aux_df.iloc[i-1], aux_df.iloc[i]) for i in range(1,aux_df.shape[0])])


            temp_df = pd.DataFrame(data = np.cumsum(euc_dis), columns = ['distance'])

            trajectory_id = np.array([i for j in range(1,aux_df.shape[0])])
            temp_df['traj'] = trajectory_id

            path_lenghts = pd.concat([path_lenghts,temp_df], ignore_index=True)
                
            
        return path_lenghts
    
    
    
    def get_trajectories_msd(self, trajectories, traj_df):
        
        msd_df = pd.DataFrame(columns=['MSD','traj'])
        
        for i in range(n_traj):
            aux_df= bm_2d_df.loc[bm_2d_df.traj == i, ['x','y']]
            msd = np.empty(shape=(0))

            for tau in range(1,aux_df.shape[0]):
                BM_msd = get_msd(tau,aux_df)

                msd = np.append(msd,BM_msd)

            temp_df = pd.DataFrame(data = msd, columns = ['MSD'])

            trajectory_id = np.array([i for j in range(1,aux_df.shape[0])])
            temp_df['traj'] = trajectory_id

            msd_df = pd.concat([msd_df,temp_df], ignore_index=True)
                
        
        return msd_df
    
    
    
    def get_ta_distribution(self, traj_df):
        
        ta_dist = np.empty(shape=(0))

    
        for index, row in traj_df[1:-1].iterrows():
            
            turning_angle = get_turning_angle(traj_df.iloc[index-1],traj_df.iloc[index],traj_df.iloc[index+1])
            ta_dist = np.append(ta_dist,turning_angle)
        
        return ta_dist
    
    
    
    def get_sl_distribution(self, traj_df):
        
            ta = self.get_ta_distribution(traj_df)
            
            ta_df= pd.DataFrame(data = ta, columns=['TA_Levy'])
            
            
            sl_array = np.empty(shape=(0))
            
            for group, elements in ta_df.groupby((ta_df['TA_Levy'].shift()!= ta_df['TA_Levy']).cumsum()):
                if elements.shape[0]>1:
                    sl_array = np.append(sl_array,elements.shape[0]+1)
                    
            return sl_array

time: 15 ms (started: 2022-09-23 23:01:49 -05:00)


## Auxiliar functions

### Euclidean distance

$d_E(p,q)=\sqrt{(p_x-q_x)^2+(p_y-q_y)^2}$

In [7]:
def get_euclidean_distance(p,q):
    """
        Arguments:
            p: [x,y] values for the starting point
            q: [x,y] values for the ending point
    """
  
    distance = np.sqrt(np.square(p[0]-q[0]) + np.square(p[1]-q[1]))

    return distance

time: 0 ns (started: 2022-09-23 23:01:49 -05:00)


### Mean Square Displacement


$MSD = \frac{1}{N-n} \sum \limits_{i=1}^{N-n}(\vec{r}_{i+n}-\vec{r}_i)^2 \quad\quad n=1,...,N-1$
<br><br>

$MSD = \frac{1}{N-n}\sum \limits_{i=1}^{N-n}{d_E(p,q)}^2 \quad\quad n=1,...,N-1$

In [8]:
def get_msd(tau,path):
    """
      Arguments:
        tau:
        path:
    """

    square_displacement = 0 

    for i in range(tau,path.shape[0],1):
        square_displacement += np.square(get_euclidean_distance(path.iloc[i-tau], path.iloc[i]))

    msd= (1/(path.shape[0]-tau))*square_displacement

    return msd

time: 0 ns (started: 2022-09-23 23:01:49 -05:00)


### Turning Angle

$
tan(\phi)=\frac{|\vec{p}\times\vec{q}|}{\vec{p}\cdot\vec{q}}
$

In [9]:
def get_turning_angle(a,b,c,round_to_zero = False):
    """
      Arguments:
        a: coordinates for p vector's tail
        b: coordinates p vector's head / q vector's tail
        c: coordinates q vector's head
        round_to_zero: if true checks if value is close enough to zero to be take as it
     """

    p = np.subtract(b,a)
    q = np.subtract(c,b)

  
    pq_cross = np.cross(p,q)
    pq_scalar = np.dot(p,q)

    phi_angle = np.arctan2(pq_cross,pq_scalar)

    if round_to_zero:
        if  count_like_zero(phi_angle):
            phi_angle = 0
      
    return phi_angle

time: 0 ns (started: 2022-09-23 23:01:49 -05:00)


### Custom round

In [10]:
def count_like_zero(number):
    """
        Arguments:
            number: 
        Return: True if -0.009 <= number <= 0.009 False otherwise
    """
    is_zero = False

    if  ((number >= -0.009) & (number <= 0)) | ((number >= 0) & (number <= 0.009)):
        is_zero = True

    return is_zero

time: 0 ns (started: 2022-09-23 23:01:49 -05:00)


### Is Negligible turn

In [11]:
def drop_turn(angle_i, angle_j):
    """
      Arguments:
        angle_i:
        angle_j: 
      Return: True if difference between angles is less than 0.001 False otherwise
      """
    is_dropable = False
    
    if (angle_i > 0) & (angle_j > 0): 
        rest = abs(angle_i - angle_j)
        if abs(angle_i - angle_j) < 0.001:
            is_dropable = True

    return is_dropable

time: 0 ns (started: 2022-09-23 23:01:49 -05:00)


# PLOT FUNCTIONS

## Plot Trajectories

In [12]:
def plot_trajectories(trajectories_df, n_trajectories = 1, title = "", line_name_prefix="", line_name_subfix = [""]):
    
    fig_3d = go.Figure()

    for i in range(n_trajectories):
        
        subfix = lambda : line_name_subfix[i] if len(line_name_subfix) > 1 else i
        trajectory_name = line_name_prefix +": " + str(subfix())

        fig_3d.add_scatter3d(
                x = trajectories_df.loc[trajectories_df['traj']==i,'x'], 
                y = trajectories_df.loc[trajectories_df['traj']==i,'y'], 
                z = trajectories_df.index,
                marker = dict(size=2),
                line = dict(width=2),
                mode = 'lines',
                name = trajectory_name,
                showlegend = True)

    
    fig_3d.update_layout(
            title_text = title,
            width = 800,
            height = 800,
            scene_camera = dict(eye = dict(x=0, y=0, z=2.5)),
            scene = dict(
                xaxis = dict(title = 'x'),
                yaxis = dict(title = 'y'),
                zaxis = dict(title = 'time', nticks = 20)
            ))

    return fig_3d

time: 0 ns (started: 2022-09-23 23:01:49 -05:00)


## Plot Metrics

In [13]:
def plot_metrics(metrics_df, n_trajectories = 1, title = "", line_name_prefix="", line_name_subfix = [""]):
    
    fig_2d = go.Figure()
    
    for i in range(n_trajectories):
        
        subfix = lambda : line_name_subfix[i] if len(line_name_subfix) > 1 else i
        trajectory_name = line_name_prefix +": " + str(subfix())

        fig_2d.add_scatter( 
                    x = metrics_df.index,
                    y = metrics_df.loc[metrics_df['traj']==i,'MSD'], 
                    marker = dict(size=0.5),
                    line = dict(width=3),
                    mode = 'lines',
                    name = trajectory_name,
                    showlegend = True)
        
    fig_2d.update_layout(
            title_text = title,
            autosize = False)

    fig_2d.show()

time: 0 ns (started: 2022-09-23 23:01:49 -05:00)


# PRUEBAS

## Trajectory controls

In [14]:
css = '''
.bk.controls_main {
    max-width: 40vw !important;
    height: 880px !important;
    border-radius: 15px;
}

.bk.min_h_control {
    min-height: 50px !important;
    top: 50;
}


.bk.hide_blocks {
}

.bk.show_blocks {
    min-height: 200px !important;
}

.bk.max_38_vwh{
  max-width:38vw !important;
}

.bk.max_50_px{
  max-width:80px !important;
  margin-right: 0;
}

'''

pn.extension(raw_css=[css])

time: 16 ms (started: 2022-09-23 23:01:49 -05:00)


In [15]:
traj_type = pnw.RadioButtonGroup(
    name='Trajectory type', 
    options=['Brownian motion','Correlated Random Walk', 'Levy Flight'], 
    button_type = 'primary', margin=(15,0,0,5))

time: 0 ns (started: 2022-09-23 23:01:49 -05:00)


## Basic Controls

In [16]:
n_traj = pnw.IntSlider(name='Numer of trajectories', value=1, step=1, start=1, end=3,css_classes = ['max_38_vwh'])

s_x = pnw.IntInput(name='X Starting position',value=0, step=1, start=-100, end=100, width = 100)
s_y = pnw.IntInput(name='Y Starting position',value=0, step=1, start=-100, end=100, width = 100)

speed = pnw.IntInput(name='Speed',value=6, step=1, start=1, end=10, width=100)

n_steps = pnw.IntSlider(name='Numer of steps', value=1000, step=1000, start=1000, end=100000)

time: 0 ns (started: 2022-09-23 23:01:49 -05:00)


In [17]:
basic_controls = pn.Column(
    n_traj,
    '###### Max 3 trayectories',
    '#### General parameters',
    '###### all trajectories will be set with same general satrting parameters',
    pn.Row(s_x,s_y),
    pn.Row(speed,n_steps))

time: 0 ns (started: 2022-09-23 23:01:49 -05:00)


## BM Controls

In [18]:
bm_type = pnw.RadioButtonGroup(name='BM Type', options=['BM 2D', 'North-South-East-Weast Restriction'], button_type = 'default')

time: 0 ns (started: 2022-09-23 23:01:49 -05:00)


## CRW Controls

In [19]:
crw_e_1 = pnw.FloatInput(name='c',value=0.6, step=0.1, start=0.1, end=0.9, width=80)
crw_e_2 = pnw.FloatInput(name='c for t2',value=0.6, step=0.1, start=0.1, end=0.9, width=80)
crw_e_3 = pnw.FloatInput(name='c for t3',value=0.6, step=0.1, start=0.1, end=0.9, width=80)
crw_e_4 = pnw.FloatInput(name='c for t4',value=0.6, step=0.1, start=0.1, end=0.9, width=80)

time: 0 ns (started: 2022-09-23 23:01:49 -05:00)


In [20]:
apply_crwe_to_all = pnw.RadioButtonGroup(name='crwe_apply', options=['YES', 'NO'], button_type = 'default')

time: 0 ns (started: 2022-09-23 23:01:49 -05:00)


In [21]:
CRW_controls = pn.Column(
    '#### CRW Parameters',
    pn.Column('##### Apply same exponent to all trajectories?',apply_crwe_to_all),
    crw_e_1)

time: 0 ns (started: 2022-09-23 23:01:49 -05:00)


## Levy controls

In [22]:
levy_alpha = pnw.FloatInput(name='alpha',value=1, step=0.1, start=0, end=2, width= 80)
levy_beta = pnw.FloatInput(name='beta',value=1, step=0.1, start=0, end=1, width= 80)
levy_loc = pnw.IntSlider(name='loc', value=3, step=1, start=0, end=10, width= 80)

time: 15 ms (started: 2022-09-23 23:01:49 -05:00)


In [23]:
Levy_controls = pn.Column(
    '#### Levy values',
    pn.Row(levy_alpha,levy_beta, levy_loc))

time: 0 ns (started: 2022-09-23 23:01:49 -05:00)


## ALL

In [43]:
@pn.depends(traj_type, n_traj, apply_crwe_to_all)
def display_controls(traj_type, n_traj, apply_crwe_to_all):
    
    controls_layout = pn.Column('#### hola')
    
    if traj_type == "Brownian motion":
        controls_layout = pn.Column(basic_controls, '#### Brownian Motion Type', bm_type)
        
        
    if traj_type =='Correlated Random Walk' and  n_traj == 2 and apply_crwe_to_all == 'NO':
        controls_layout = pn.Column(basic_controls, CRW_controls, crw_e_2)
    
    if traj_type =='Correlated Random Walk' and  n_traj == 3 and apply_crwe_to_all == 'NO':
        controls_layout = pn.Column(basic_controls, CRW_controls, pn.Row(crw_e_2,crw_e_3))
    
    if traj_type =='Correlated Random Walk' and  (n_traj == 1 or apply_crwe_to_all == 'YES'):
        controls_layout = pn.Column(basic_controls, CRW_controls)
        
        
    if traj_type == "Levy Flight":
        controls_layout = pn.Column(basic_controls,Levy_controls)
        
    return controls_layout

time: 0 ns (started: 2022-09-23 23:13:35 -05:00)


In [55]:
@pn.depends(traj_type, n_traj, s_x, s_y, speed, n_steps, bm_type, apply_crwe_to_all, crw_e_1, crw_e_2, crw_e_3, levy_alpha, levy_beta, levy_loc)
def plot_traj(traj_type, n_traj, s_x, s_y, speed, n_steps,  bm_type, apply_crwe_to_all, crw_e_1, crw_e_2, crw_e_3, levy_alpha, levy_beta, levy_loc):
    
    
    # define trajectory type
    tt = ""
    prefix = "trayectory_"
    if traj_type == "Brownian motion" and bm_type == 'BM 2D' :
        tt = "BM_2d"
        
    if traj_type == "Brownian motion" and bm_type == 'North-South-East-Weast Restriction' :
        tt = "BM_nsew"
        
    if traj_type == "Correlated Random Walk":
        tt = "CRW"
        prefix = "cuachy_"
        
        
    exponents = [0.1]      
    if traj_type =='Correlated Random Walk' and (n_traj == 1 or apply_crwe_to_all == 'YES'):
        exponents = [crw_e_1]
        
    if traj_type =='Correlated Random Walk' and n_traj == 2 and apply_crwe_to_all == 'NO':
        exponents = [crw_e_1, crw_e_2]
    
    if traj_type =='Correlated Random Walk' and n_traj == 3 and apply_crwe_to_all == 'NO':
         exponents = [crw_e_1, crw_e_2, crw_e_3]
    
    
        
        
            
    if not traj_type == "Levy Flight":
        explorers = Explorers(s_x,s_y)
        traj = explorers.trace_trajectories(n_traj = n_traj, traj_type= tt, n_steps = n_steps, speed = speed, crw_exponents = exponents)
    
    if traj_type == "Levy Flight":
        tt = "LF"
        prefix = "Levy_alpha_"
        levy_flight = LFlyTrajectory (s_x,s_y)
        traj = levy_flight.levy_walk(alpha = levy_alpha, beta = levy_beta, loc = levy_loc, samples = n_steps)
    
    fig =  plot_trajectories(traj,n_traj,traj_type, prefix)

    return fig


time: 0 ns (started: 2022-09-23 23:15:02 -05:00)


In [56]:
pn.Row(pn.Column(traj_type,display_controls, background = "WhiteSmoke"),  pn.Row(plot_traj, "# Metrcis"))

Row
    [0] Column(background='WhiteSmoke')
        [0] RadioButtonGroup(button_type='primary', margin=(15, 0, 0, 5), name='Trajectory type', options=['Brownian motion', ...], value='Correlated Random Walk')
        [1] ParamFunction(function)
    [1] Row
        [0] ParamFunction(function)
        [1] Markdown(str)

time: 188 ms (started: 2022-09-23 23:15:07 -05:00)
